In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Pre-processing and setup functions
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.feature_selection import mutual_info_classif

# PCA Decomposition
from sklearn.decomposition import PCA

# Neural Network
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Set viewable columns in pandas
pd.set_option('display.max_columns', 50)

In [2]:
exo2 = pd.read_csv('transformed_features.csv')

In [3]:
exo2 = exo2.drop(columns=['Unnamed: 0'])

## Transform and Encode

In [4]:
# Separate X and y subsets
y = exo2['koi_disposition']
X = exo2.drop(['koi_disposition'], axis=1)

# Split X and y subsets into Test and Train datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Scale continuous values between 0 < x < 1
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Transform the y subsets into model usable representations
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

C:\Users\TPCal\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## Build Model

In [5]:
model = Sequential()

model.add(Dense(
    units = 100, kernel_regularizer = keras.regularizers.l2(0.001),
    activation = 'relu', 
    input_dim = 67
))

model.add(Dense(
    units = 100, kernel_regularizer = keras.regularizers.l2(0.001),
    activation = 'relu'
))

model.add(Dense(
    units = 100, kernel_regularizer = keras.regularizers.l2(0.001),
    activation = 'relu'
))

model.add(Dense(
    units = 100, kernel_regularizer = keras.regularizers.l2(0.001),
    activation = 'relu'
))

model.add(Dense(
    units = 100, kernel_regularizer = keras.regularizers.l2(0.001),
    activation = 'relu'
))

model.add(Dense(
    units = 3, 
    activation = 'softmax'
))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               6800      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 47,503
Trainable params: 47,503
Non-trainable params: 0
_________________________________________________________________


## Compile and Fit the Model

In [7]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(
    X_train_scaled, 
    y_train_categorical, 
    epochs = 40, 
    shuffle = True, 
    verbose = 1
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
5243/5243 [==============================] - 1s 115us/sample - loss: 0.7571 - acc: 0.7583
Epoch 2/40
5243/5243 [==============================] - 0s 58us/sample - loss: 0.5066 - acc: 0.8488
Epoch 3/40
5243/5243 [==============================] - 0s 58us/sample - loss: 0.4736 - acc: 0.8516
Epoch 4/40
5243/5243 [==============================] - 0s 61us/sample - loss: 0.4370 - acc: 0.8709
Epoch 5/40
5243/5243 [==============================] - 0s 57us/sample - loss: 0.4160 - acc: 0.8722
Epoch 6/40
5243/5243 [==============================] - 0s 62us/sample - loss: 0.4089 - acc: 0.8751
Epoch 7/40
5243/5243 [==============================] - 0s 65us/sample - loss: 0.3978 - acc: 0.8747
Epoch 8/40
5243/5243 [==============================] - 0s 60us/sample - loss: 0.3940 - acc: 0.8716
Epoch 9/40
5243/5243 [==============================] - 0s 58us/sample - loss: 0.3725 - acc: 0.8850
Epoch 10/40
5243/5243 [============================

## Model Accuracy

In [8]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=0)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.30301057828645683, Accuracy: 0.8947368264198303


In [9]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

print(f'Predicted classes: {prediction_labels}')
print(f'Actual labels: {list(y_test[:10])}')

Predicted classes: ['CONFIRMED' 'CONFIRMED' 'CONFIRMED' 'CONFIRMED' 'FALSE POSITIVE'
 'FALSE POSITIVE' 'CONFIRMED' 'CANDIDATE' 'CONFIRMED' 'FALSE POSITIVE']
Actual labels: ['CONFIRMED', 'CONFIRMED', 'CONFIRMED', 'CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE']


In [10]:
model.save('Deep_NN.h5')